# Part 3: Deep Dive - Related Work 생성 (LitLLM)

**SNU AI Psychology Workshop - February 2026**

---

## 학습 목표
1. LitLLM의 코드 구조와 데이터 흐름 이해
2. 본인 초록으로 Related Work 초안 생성
3. 프롬프트/스타일 커스텀으로 출력 품질 개선

---
## Setup

In [ ]:
# Cell 1: 환경 설정 (Colab/로컬 자동 감지)
import os
import sys

# Colab 환경 감지
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    WORKSHOP_DIR = "/content/drive/MyDrive/aiworkshop_Feb2026/"
    os.makedirs(WORKSHOP_DIR, exist_ok=True)
    os.chdir(WORKSHOP_DIR)
    IN_COLAB = True
    print("🌐 Colab 환경에서 실행 중")
except ImportError:
    # 로컬 환경
    WORKSHOP_DIR = os.getcwd()
    IN_COLAB = False
    print("💻 로컬 환경에서 실행 중")

print(f"작업 폴더: {WORKSHOP_DIR}")

In [ ]:
# Cell 2: 패키지 설치
# Colab: 자동 설치
# 로컬: pyproject.toml로 미리 설치 필요 (uv pip install -e . 또는 pip install -e .)

if IN_COLAB:
    print("📦 패키지 설치 중...")
    %pip install google-generativeai python-dotenv requests semanticscholar -q
    print("✅ 설치 완료!")
else:
    print("✅ 로컬 환경: pyproject.toml로 설치된 패키지 사용")
    print("   (미설치 시: uv pip install -e . 또는 pip install -e .)")

print("\n💡 LitLLM 웹 데모: https://litllm.onrender.com")
print("   이 노트북은 LitLLM 방식을 재현한 간소화 버전입니다.")

In [ ]:
# Cell 3: API Key 로딩
import os

if IN_COLAB:
    try:
        from google.colab import userdata
        OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
        GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
        print("✅ Colab Secrets에서 API Key 로딩 완료")
    except:
        OPENAI_API_KEY = None
        GEMINI_API_KEY = None
        print("⚠️ Colab Secrets에 API KEY를 추가하세요")
else:
    try:
        from dotenv import load_dotenv
        load_dotenv()
        OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
        GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
        if OPENAI_API_KEY or GEMINI_API_KEY:
            print("✅ .env 파일에서 API Key 로딩 완료")
        else:
            print("⚠️ .env 파일에 API KEY를 추가하세요")
    except ImportError:
        OPENAI_API_KEY = None
        GEMINI_API_KEY = None

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY or ''
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY or ''

print(f"\n🔑 OpenAI: {'설정됨' if OPENAI_API_KEY else '없음 (LLM 기능 제한)'}")
print(f"🔑 Gemini: {'설정됨' if GEMINI_API_KEY else '없음 (LLM 기능 제한)'}")
print("💡 Semantic Scholar 검색은 API key 없이도 사용 가능합니다")

---
## 1. 파악하기: LitLLM 코드 구조

### LitLLM 구조 (TMLR 2024)

LitLLM은 웹 애플리케이션으로 제공됩니다: https://litllm.onrender.com

**핵심 파이프라인 (이 노트북에서 재현):**

```
1. Retriever  → 관련 논문 검색 (Semantic Scholar)
2. Summarizer → 논문 요약 (LLM)
3. Writer     → Related Work 생성 (LLM)
```

**워크플로우:**
1. 논문 초록 입력 → 키워드 추출
2. `retriever`가 Semantic Scholar에서 관련 논문 검색
3. `summarizer`가 각 논문 요약
4. `writer`가 Related Work 문단 생성 (인용 포함)

In [ ]:
# Cell 7: 핵심 모듈 직접 구현 (LitLLM 스타일)
import requests
import google.generativeai as genai

class LitLLMPipeline:
    """
    LitLLM 핵심 파이프라인 재현
    retriever → summarizer → writer
    """
    
    def __init__(self):
        if GEMINI_API_KEY:
            genai.configure(api_key=GEMINI_API_KEY)
            self.model = genai.GenerativeModel('gemini-1.5-flash')
        else:
            self.model = None
    
    def retrieve_papers(self, abstract, num_papers=5):
        """Step 1: 관련 논문 검색 (Semantic Scholar)"""
        # 키워드 추출 (간단 버전)
        keywords = self._extract_keywords(abstract)
        
        # Semantic Scholar 검색
        url = "https://api.semanticscholar.org/graph/v1/paper/search"
        params = {
            "query": " ".join(keywords),
            "limit": num_papers,
            "fields": "title,authors,year,abstract,citationCount"
        }
        response = requests.get(url, params=params)
        return response.json().get('data', [])
    
    def summarize_paper(self, paper):
        """Step 2: 논문 요약 (LLM)"""
        if not self.model:
            return f"{paper['title']}: {paper.get('abstract', '')[:100]}..."
        
        prompt = f"""Summarize this paper in 1-2 sentences for a Related Work section:

Title: {paper['title']}
Abstract: {paper.get('abstract', 'N/A')}

Focus on the main contribution and methodology."""
        
        response = self.model.generate_content(prompt)
        return response.text
    
    def write_related_work(self, abstract, summaries):
        """Step 3: Related Work 생성 (LLM)"""
        if not self.model:
            return "LLM API key 필요"
        
        papers_text = "\n".join([f"- {s}" for s in summaries])
        
        prompt = f"""Write a Related Work section for a paper with this abstract:

ABSTRACT:
{abstract}

RELATED PAPERS:
{papers_text}

Requirements:
- Write 2-3 paragraphs
- Group papers by theme
- Use academic writing style
- Include citations as (Author et al., Year)"""
        
        response = self.model.generate_content(prompt)
        return response.text
    
    def _extract_keywords(self, text):
        """간단한 키워드 추출"""
        # 불용어 제거 후 빈도순 정렬 (간단 버전)
        stopwords = {'the', 'a', 'an', 'is', 'are', 'was', 'were', 'be', 'been', 
                     'being', 'have', 'has', 'had', 'do', 'does', 'did', 'will',
                     'would', 'could', 'should', 'may', 'might', 'must', 'shall',
                     'to', 'of', 'in', 'for', 'on', 'with', 'at', 'by', 'from',
                     'as', 'into', 'through', 'during', 'before', 'after', 'and',
                     'but', 'or', 'nor', 'so', 'yet', 'both', 'either', 'neither',
                     'not', 'only', 'own', 'same', 'than', 'too', 'very', 'just',
                     'this', 'that', 'these', 'those', 'we', 'our', 'they', 'their'}
        
        words = text.lower().split()
        keywords = [w for w in words if w.isalpha() and w not in stopwords and len(w) > 3]
        
        # 빈도 계산
        from collections import Counter
        freq = Counter(keywords)
        return [w for w, _ in freq.most_common(5)]

print("✅ LitLLMPipeline 클래스 정의 완료 (Gemini 사용)")
print("   메서드: retrieve_papers(), summarize_paper(), write_related_work()")

### 질문

LitLLM의 3단계 파이프라인을 보고 생각해보세요:
- 어떤 단계에서 품질 손실이 가장 클까?
- 어떤 프롬프트를 수정하면 결과가 좋아질까?

---
## 2. 써보기: Related Work 생성

### 샘플 데이터

테스트할 논문 초록 (AI 심리학 관련)

In [ ]:
# Cell 10: 샘플 실행 (발표자 데모)
sample_abstract = """
Large language models (LLMs) have shown remarkable capabilities in various 
cognitive tasks. This paper investigates whether LLMs exhibit human-like 
cognitive biases, specifically focusing on anchoring effects and confirmation 
bias. We design a series of experiments adapted from classical psychology 
studies and evaluate GPT-4 and Claude on these tasks. Our results show that 
LLMs do exhibit similar biases to humans, but with different magnitudes. 
This has implications for using LLMs as cognitive models and for AI safety.
"""

pipeline = LitLLMPipeline()

# Step 1: 논문 검색
print("🔍 Step 1: 관련 논문 검색...")
papers = pipeline.retrieve_papers(sample_abstract, num_papers=5)
print(f"   {len(papers)}개 논문 발견\n")

for p in papers[:3]:
    print(f"   - {p['title'][:60]}...")

In [ ]:
# Cell 11: 요약 + Related Work 생성
if GEMINI_API_KEY and papers:
    # Step 2: 각 논문 요약
    print("📝 Step 2: 논문 요약...")
    summaries = []
    for paper in papers[:5]:
        summary = pipeline.summarize_paper(paper)
        summaries.append(summary)
        print(f"   ✓ {paper['title'][:40]}...")
    
    # Step 3: Related Work 생성
    print("\n✍️ Step 3: Related Work 생성...\n")
    related_work = pipeline.write_related_work(sample_abstract, summaries)
    
    print("=" * 60)
    print("RELATED WORK")
    print("=" * 60)
    print(related_work)
else:
    print("Gemini API key 없음 - 요약/생성 단계 건너뜀")

### DIY: 본인 초록으로 Related Work 생성

In [ ]:
# DIY Cell 13: 본인 초록 입력
# 힌트: my_abstract 변수에 본인 논문 초록을 붙여넣으세요

my_abstract = """
여기에 본인 논문 초록을 붙여넣으세요.
또는 연구 아이디어를 2-3문장으로 작성하세요.
"""

# 실행
print("🔍 논문 검색 중...")
my_papers = pipeline.retrieve_papers(my_abstract, num_papers=5)
print(f"검색 결과: {len(my_papers)}개\n")

for p in my_papers:
    print(f"- {p['title']}")
    print(f"  ({p.get('year', 'N/A')}, 인용: {p.get('citationCount', 0)})")

In [ ]:
# DIY Cell 14: Related Work 생성
if GEMINI_API_KEY and my_papers:
    print("📝 요약 중...")
    my_summaries = [pipeline.summarize_paper(p) for p in my_papers[:5]]
    
    print("\n✍️ Related Work 생성 중...\n")
    my_related_work = pipeline.write_related_work(my_abstract, my_summaries)
    
    print("=" * 60)
    print("MY RELATED WORK")
    print("=" * 60)
    print(my_related_work)
else:
    print("Gemini API key가 필요합니다.")

---
## 3. 바꿔보기: 프롬프트 커스텀

### 수정 포인트

| 항목 | 기본값 | 커스텀 아이디어 |
|------|--------|----------------|
| 요약 스타일 | 일반 요약 | "방법론 중심으로", "결과 중심으로" |
| 작성 스타일 | 일반 학술체 | "APA 스타일", "비판적 톤" |
| 언어 | 영어 | 한글 |
| 길이 | 2-3 문단 | 1 문단, 5 문단 |

In [ ]:
# Cell 16: 프롬프트 수정 예시 (Before/After)

# Before: 기본 요약 프롬프트
SUMMARY_PROMPT_BASIC = """Summarize this paper in 1-2 sentences.
Title: {title}
Abstract: {abstract}"""

# After: 방법론 중심 요약 프롬프트
SUMMARY_PROMPT_METHOD = """Summarize this paper's METHODOLOGY in 1-2 sentences.
Focus on: experimental design, dataset, evaluation metrics.

Title: {title}
Abstract: {abstract}

Format: "[Author] used [method] to [goal], evaluating on [dataset]."""

# After: 결과 중심 요약 프롬프트
SUMMARY_PROMPT_RESULTS = """Summarize this paper's KEY FINDINGS in 1-2 sentences.
Focus on: main results, performance numbers, implications.

Title: {title}
Abstract: {abstract}

Format: "[Author] found that [finding], achieving [result]."""

print("프롬프트 템플릿 정의 완료")
print("- SUMMARY_PROMPT_BASIC: 일반 요약")
print("- SUMMARY_PROMPT_METHOD: 방법론 중심")
print("- SUMMARY_PROMPT_RESULTS: 결과 중심")

In [ ]:
# DIY Cell 17: 본인만의 프롬프트 작성

MY_SUMMARY_PROMPT = """
# TODO: 본인 연구에 맞는 요약 프롬프트 작성

아이디어:
- "심리학 관점에서 요약"
- "임상 적용 가능성 중심으로"
- "한계점 포함해서 요약"

Title: {title}
Abstract: {abstract}
"""

MY_WRITER_PROMPT = """
# TODO: 본인 스타일에 맞는 Related Work 작성 프롬프트

아이디어:
- "APA 7th 스타일로"
- "비판적 톤으로 (각 논문의 한계점 언급)"
- "한글로 작성"
"""

In [ ]:
# Cell 18: 한글 Related Work 생성 예시
import google.generativeai as genai

KOREAN_WRITER_PROMPT = """다음 논문 초록을 위한 '선행연구' 섹션을 한글로 작성하세요.

초록:
{abstract}

관련 논문 요약:
{summaries}

작성 요구사항:
- 2-3개 문단으로 구성
- 주제별로 논문 그룹화
- 학술 논문 스타일 (존댓말 X)
- 인용은 (저자, 연도) 형식
- 마지막에 본 연구와의 차별점 1-2문장 추가"""

def write_korean_related_work(abstract, summaries):
    if not GEMINI_API_KEY:
        return "Gemini API key 필요"
    
    genai.configure(api_key=GEMINI_API_KEY)
    model = genai.GenerativeModel('gemini-1.5-flash')
    papers_text = "\n".join([f"- {s}" for s in summaries])
    
    prompt = KOREAN_WRITER_PROMPT.format(
        abstract=abstract, 
        summaries=papers_text
    )
    
    response = model.generate_content(prompt)
    return response.text

# 테스트
if GEMINI_API_KEY and 'summaries' in dir():
    korean_rw = write_korean_related_work(sample_abstract, summaries)
    print("=== 한글 선행연구 ===")
    print(korean_rw)
else:
    print("이전 셀에서 summaries를 먼저 생성하세요.")

In [ ]:
# DIY Cell 19: 스타일 비교 실험

# 다양한 스타일로 Related Work 생성해보기
styles = {
    "기본": "Write a Related Work section in academic style.",
    "비판적": "Write a critical Related Work section, pointing out limitations of each work.",
    "간결": "Write a concise Related Work section in exactly 3 sentences."
}

# TODO: 각 스타일로 생성하고 비교해보세요
# for style_name, style_prompt in styles.items():
#     result = generate_with_style(abstract, summaries, style_prompt)
#     print(f"=== {style_name} ===")
#     print(result)

In [ ]:
# Cell 20: 결과 비교 (원본 vs 커스텀)
print("""
📊 프롬프트 스타일별 결과 비교

| 스타일 | 장점 | 단점 | 추천 용도 |
|--------|------|------|----------|
| 기본 | 무난함 | 특색 없음 | 초안 작성 |
| 방법론 중심 | 기술적 디테일 | 이론 부족 | 실험 논문 |
| 비판적 | 차별점 명확 | 너무 부정적 | 리뷰 논문 |
| 한글 | 직접 사용 가능 | 인용 형식 확인 필요 | 국내 학회 |

💡 팁: 여러 스타일의 결과를 조합하면 더 좋은 초안이 됩니다.
""")

---
## 4. 토론

### 토론 질문

1. **생성된 Related Work의 품질은 어떤가요?**
   - 인용이 정확한가?
   - 논리적 흐름이 자연스러운가?
   - 어떤 부분을 수정해야 할까?

2. **어떤 프롬프트가 가장 효과적이었나요?**
   - 기본 vs 방법론 중심 vs 비판적
   - 영어 vs 한글

3. **실제 논문 작성에 어떻게 활용할 수 있을까?**
   - 초안으로 시작해서 수정?
   - 빠진 논문 추가 방법?
   - 인용 형식 자동 변환?

### 다음 단계

이어서 실습할 내용:
- **Part 4**: AgentReview로 작성한 논문에 AI 피드백 받기

### 과제 아이디어
```python
# PubMed 검색 추가 (심리학 논문용)
def retrieve_from_pubmed(abstract):
    # E-utilities API 사용
    pass

# 메타분석용 요약 템플릿
META_ANALYSIS_PROMPT = """
효과크기(d 또는 r), 샘플 수, 실험 조건을 추출하세요.
"""

# BibTeX 자동 생성
def generate_bibtex(papers):
    pass
```